In [ ]:
import pandas as pd
df=pd.read_csv('features_band.csv')

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
names = ["Nearest Neighbors", "Linear SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "AdaBoost",
         "Naive Bayes"]
classifiers = [
    KNeighborsClassifier(5),
    SVC(kernel="linear", C=0.025),
    GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
    DecisionTreeClassifier(max_depth=3),
    RandomForestClassifier(max_depth=3, n_estimators=10),
    AdaBoostClassifier(),
    ]


In [ ]:
cpc_values = df['CPC'].values

# Normalize EEG data (110 columns)
eeg_data = df.iloc[:, 7:81].values  # these columns are EEG data
scaler_eeg = MinMaxScaler()
eeg_data_normalized = scaler_eeg.fit_transform(eeg_data)
static_features = df.iloc[:, :7].values
# Normalize static features
scaler_static = MinMaxScaler()
static_features_normalized = scaler_static.fit_transform(static_features)
# Combine normalized static features, normalized EEG data, and CPC values
normalized_data = np.concatenate((static_features_normalized, eeg_data_normalized), axis=1)

# Convert CPC values to one-hot encoded labels
one_hot_labels = to_categorical(cpc_values - 1, num_classes=5)  # Assuming CPC values start from 1
# Train-test split
train_data, test_data, train_labels, test_labels = train_test_split(
    normalized_data, one_hot_labels, test_size=0.2, random_state=42
)

In [ ]:
testlab=[]
for i in test_labels.tolist():
  testlab.append([i.index(1.0)+1])
trainlab=[]
for i in train_labels.tolist():
  trainlab.append([i.index(1.0)+1])

In [ ]:
# Define model
# Define model
model1 = Sequential()
model1.add(LSTM(units=128, input_shape=(81, 1), return_sequences=True))  # Input shape should match the number of EEG columns (117)
model1.add(Dropout(0.3))  # Add dropout layer with a dropout rate of 30%
model1.add(LSTM(units=64, return_sequences=True))
model1.add(LSTM(units=32))
model1.add(Dense(units=16, activation='relu'))
model1.add(Dense(units=5, activation='softmax'))

# Compile model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#features_band
#for 81 columns-features_dt
model1.fit(x=train_data, y=train_labels, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
13/13 [==============================] - 12s 444ms/step - loss: 1.4320 - accuracy: 0.5530 - val_loss: 1.0458 - val_accuracy: 0.6082
Epoch 2/50
13/13 [==============================] - 3s 235ms/step - loss: 1.0604 - accuracy: 0.5788 - val_loss: 0.9958 - val_accuracy: 0.6082
Epoch 3/50
13/13 [==============================] - 3s 240ms/step - loss: 1.0414 - accuracy: 0.5788 - val_loss: 0.9874 - val_accuracy: 0.6082
Epoch 4/50
13/13 [==============================] - 3s 238ms/step - loss: 1.0381 - accuracy: 0.5788 - val_loss: 0.9795 - val_accuracy: 0.6082
Epoch 5/50
13/13 [==============================] - 4s 335ms/step - loss: 1.0361 - accuracy: 0.5788 - val_loss: 0.9745 - val_accuracy: 0.6082
Epoch 6/50
13/13 [==============================] - 3s 243ms/step - loss: 1.0368 - accuracy: 0.5788 - val_loss: 0.9722 - val_accuracy: 0.6082
Epoch 7/50
13/13 [==============================] - 3s 236ms/step - loss: 1.0339 - accuracy: 0.5788 - val_loss: 0.9717 - val_accuracy: 0.6082
Epoch

In [ ]:
loss, accuracy = model1.evaluate(x=test_data, y=test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

4/4 [==============================] - 0s 67ms/step - loss: 0.2688 - accuracy: 0.8607
Test Loss: 0.2687661051750183, Test Accuracy: 0.8606557250022888


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import euclidean
from math import exp



gem_matrix_train = np.zeros((train_data.shape[0], train_data.shape[0]))
for i in range(train_data.shape[0]):
    for j in range(i, train_data.shape[0]):
        dtw_distance = np.sqrt(np.sum(np.square(train_data[i] - train_data[j])))
        gem_matrix_train[i, j] = exp(-dtw_distance ** 2)
        gem_matrix_train[j, i] = gem_matrix_train[i, j]

gem_matrix_test = np.zeros((test_data.shape[0], train_data.shape[0]))
for i in range(test_data.shape[0]):
    for j in range(train_data.shape[0]):
        dtw_distance = np.sqrt(np.sum(np.square(test_data[i] - train_data[j])))
        gem_matrix_test[i, j] = exp(-dtw_distance ** 2)

clf = SVC(kernel='precomputed')
clf.fit(gem_matrix_train, trainlab)
y_pred = clf.predict(gem_matrix_test)

accuracy = accuracy_score(testlab, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.90


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#81 columns
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
members=[model1,clf]


# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
  stackX = None
  for model in members:
    # make prediction
    if(model!=clf):
      yhat = model.predict(inputX)
      print('yhat: ',yhat)
      print(type(yhat))
    else:
      gem_matrix_test = np.zeros((inputX.shape[0], train_data.shape[0]))
      for i in range(inputX.shape[0]):
          for j in range(train_data.shape[0]):
              dtw_distance = np.sqrt(np.sum(np.square(inputX[i] - train_data[j])))
              gem_matrix_test[i, j] = exp(-dtw_distance ** 2)
      yhat = model.predict(gem_matrix_test)
      yhat1=[]
      for i in yhat.tolist():
        l=[0.0,0.0,0.0,0.0,0.0]
        l[i-1]=1.0
        yhat1.append(l)
      yhat=np.array(yhat1)
      print('yhat: ',yhat)
      print(type(yhat))
    # stack predictions into [rows, members, probabilities]
    if stackX is None:
      stackX = yhat
    else:
      stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
  stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
  return stackX

#[1 0 0 0 0]
#[0 1 0 0 0]

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
  # create dataset using ensemble
  stackedX = stacked_dataset(members, inputX)
  print('stacked: ',stackedX)
  # fit standalone model
  #model = LogisticRegression()
  #model=clf
  #model=classifiers[0]
  model=knn1
  #model=rfmodel ----->give 100 percent accuracy
  model.fit(stackedX, inputy)
  return model

# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

# fit stacked model using the ensemble
model = fit_stacked_model(members, test_data, testlab)
# evaluate model on test set
yhat = stacked_prediction(members, model, test_data)
acc = accuracy_score(testlab, yhat)
print('Stacked Test Accuracy: %.3f' % acc)



4/4 [==============================] - 3s 102ms/step
yhat:  [[9.68891982e-05 3.35637340e-03 5.89827728e-03 1.16152111e-02
  9.79033113e-01]
 [7.65519799e-05 2.53086118e-03 4.42599319e-03 8.94282386e-03
  9.84023690e-01]
 [2.06928793e-02 3.10279101e-01 5.80528855e-01 5.42895421e-02
  3.42096351e-02]
 [9.67522792e-05 3.35021294e-03 5.88381151e-03 1.15938159e-02
  9.79075253e-01]
 [8.92506287e-05 2.37224414e-03 3.69793014e-03 7.96466321e-03
  9.85875905e-01]
 [2.17432663e-01 6.18489385e-01 1.48567051e-01 1.05923982e-02
  4.91853850e-03]
 [3.18944663e-01 5.74841321e-01 9.04377252e-02 9.25908703e-03
  6.51714532e-03]
 [9.57839075e-05 2.43442948e-03 3.72009748e-03 8.07298627e-03
  9.85676587e-01]
 [9.94691253e-01 5.00813406e-03 2.59535998e-04 3.81116333e-05
  3.06111588e-06]
 [9.67617816e-05 3.35063459e-03 5.88479964e-03 1.15952808e-02
  9.79072452e-01]
 [9.67900560e-05 3.35182971e-03 5.88754099e-03 1.15993768e-02
  9.79064405e-01]
 [9.67689484e-05 3.35095939e-03 5.88556053e-03 1.15964040e-0

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


4/4 [==============================] - 0s 65ms/step
yhat:  [[9.68891982e-05 3.35637340e-03 5.89827728e-03 1.16152111e-02
  9.79033113e-01]
 [7.65519799e-05 2.53086118e-03 4.42599319e-03 8.94282386e-03
  9.84023690e-01]
 [2.06928793e-02 3.10279101e-01 5.80528855e-01 5.42895421e-02
  3.42096351e-02]
 [9.67522792e-05 3.35021294e-03 5.88381151e-03 1.15938159e-02
  9.79075253e-01]
 [8.92506287e-05 2.37224414e-03 3.69793014e-03 7.96466321e-03
  9.85875905e-01]
 [2.17432663e-01 6.18489385e-01 1.48567051e-01 1.05923982e-02
  4.91853850e-03]
 [3.18944663e-01 5.74841321e-01 9.04377252e-02 9.25908703e-03
  6.51714532e-03]
 [9.57839075e-05 2.43442948e-03 3.72009748e-03 8.07298627e-03
  9.85676587e-01]
 [9.94691253e-01 5.00813406e-03 2.59535998e-04 3.81116333e-05
  3.06111588e-06]
 [9.67617816e-05 3.35063459e-03 5.88479964e-03 1.15952808e-02
  9.79072452e-01]
 [9.67900560e-05 3.35182971e-03 5.88754099e-03 1.15993768e-02
  9.79064405e-01]
 [9.67689484e-05 3.35095939e-03 5.88556053e-03 1.15964040e-02

In [ ]:
import sklearn
print(sklearn.metrics.mean_squared_error(testlab,yhat))
print(sklearn.metrics.mean_absolute_error(testlab,yhat))
print(sklearn.metrics.f1_score(testlab,yhat,average='weighted'))

0.11475409836065574
0.06557377049180328
0.9440784964797895
